# DATASET

*filename:* idiom_all_repository_all.json

## schema and sample row

|variations |idiom|sources |entry | id | confidence |
|---------------|-----------------|-----------------|----------------|--------|--------|
| *varchar[] | *varchar* | *varchar[]* | *json (varchar in df)* | *int64* |*int64* |
| [] | 'er indoors | [Wiktionary] | [{"usages":[[]],"definition":"one's wife","pos":"noun"}]    | 1 | 1

### variations 
list of phrase variations of idiom
- varchar []
- can be empty list []

### idiom
the phrase
- varchar
- can NOT be NULL
  
### sources
list of sources for idiom
- varchar []
- can NOT be NULL
- possible sources: Wiktionary, Learn English Today, Oxford Dictionary of Idioms

### entry
includes list of {} with usage, defintion, and pos (part of speech tag)
- json (for og), varchar (for dataframe)
- can NOT be NULL
- can be "" or []
- "" possible with Oxford Dictionary of Idioms AND/OR Learn English Today
- [] possible with inclusion of Wiktionary

### id
unique identifier for entry/row
- int64
- can NOT be NULL
- min: 1
- max: 9878
- \# of unique values: 9721
  
### confidence
dataset confidence in idiom information
- int64
- can NOT be NULL
- possible values (in curr. dataset): [1, 2, 3]
- seems to correspond to length of sources list (only exception id# 6674)

## size
- 9721  rows
- 4.9 MB

## NOTES
- 9721 rows but 9684 unique rows excluding id
- 37 idioms with duplicates (concurs with 9721 rows - 37 duplicates = 9684 unique rows)

In [17]:
import duckdb as db

FILE_PATH = '../Data/idiom_repository_all.json'

idiom_df = db.query(f"""
    SELECT *
    FROM read_json_auto('{FILE_PATH}')
""").df()

In [19]:
db.query(f"""
    SELECT *
    FROM read_json_auto('{FILE_PATH}')
""")

┌────────────┬─────────────────────────────────┬───────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────┬────────────┐
│ variations │              idiom              │        sources        │                                                                                                                                                                                                                                                                       entry                                  

In [18]:
idiom_df

,variations,idiom,sources,entry,id,confidence
0,[],'er indoors,[Wiktionary],"[{""usages"":[[]],""definition"":""one's wife"",""pos...",1,1
1,[],10 Downing Street,[Wiktionary],"[{""usages"":[[""... explicitly clear, in the way...",2,1
2,[],11 Downing Street,[Wiktionary],"[{""usages"":[[]],""definition"":""w treasury of th...",3,1
3,[],110 proof,[Wiktionary],"[{""usages"":[[""in those days a hollywood hero h...",4,1
4,[],12-ounce curls,[Wiktionary],"[{""usages"":[[""don't burn out your biceps with ...",5,1
...,...,...,...,...,...,...
9716,[],words fail me,[Learn English Today],"""""",9874,1
9717,[],bandit territory,[Learn English Today],"""""",9875,1
9718,[],beat black and blue,[Learn English Today],"""""",9876,1
9719,[],free-for-all,[Learn English Today],"""""",9877,1


In [ ]:
# number of unique datapoints

# 9721 rows
print("number of rows")
db.query(f"""
    SELECT count(*) as num_rows
    FROM idiom_df
""").show()

# 9684 rows
print("unique rows, excluding ID")
db.query(f"""
    SELECT DISTINCT idiom, variations, sources, entry, confidence, count(*) as num_rows
    FROM idiom_df
    GROUP BY idiom, variations, sources, entry, confidence
""").show()

# 9684 rows
print("unique idioms")
db.query(f"""
    SELECT DISTINCT idiom
    FROM idiom_df
""").show()

## at most, 1 additional duplicate (max is 2 total for a single idiom phtase)
# 37 idioms with duplicates
print(f"duplicate idioms")
db.query(f"""
    SELECT DISTINCT idiom, count(*) as num
    FROM idiom_df
    GROUP BY idiom
    HAVING num > 1
    ORDER BY count(*) DESC
""").show()

# duplicates
db.query(f"""
SELECT
  df1.id AS id1,
  df2.id AS id2,

  df1.idiom,

  df1.sources AS sources1,
  df2.sources AS sources2,
  (df1.sources = df2.sources) AS sources_match,

  df1.entry AS entry1,
  df2.entry AS entry2,
  (df1.entry = df2.entry) AS entry_match,

  df1.confidence AS confidence1,
  df2.confidence AS confidence2,
  (df1.confidence = df2.confidence) AS confidence_match
FROM idiom_df df1
JOIN idiom_df df2
  ON df1.idiom = df2.idiom
WHERE df1.id < df2.id
  -- uncomment to show only cases where at least one field differs
  -- AND (
  --   coalesce(df1.sources = df2.sources, FALSE) = FALSE
  --   OR coalesce(df1.entry = df2.entry, FALSE) = FALSE
  --   OR coalesce(df1.confidence = df2.confidence, FALSE) = FALSE
  -- )
""").show()

duplicates = db.query(f"""
    WITH cte AS (
        SELECT df1.id as id1, df2.id as id2, (df1.id == df2.id) AS id_match, 
         df1.idiom as idiom, 
         df1.sources AS sources1, df2.sources AS sources2, (df1.sources == df2.sources) AS sources_match,
          df1.entry AS entry1, df2.entry AS entry2, (df1.entry == df2.entry) AS entry_match, 
         df1.confidence AS confidence1, df2.confidence AS confidence2, (df1.confidence == df2.confidence) AS confidence_match
    FROM idiom_df df1
    JOIN idiom_df df2 ON df1.idiom == df2.idiom
    WHERE df1.id != df2.id 
)
        SELECT idiom, [id1, id2] AS ids
        FROM cte
""")




number of rows
┌──────────┐
│ num_rows │
│  int64   │
├──────────┤
│     9721 │
└──────────┘

unique rows, excluding ID
┌────────────────────────────────────┬────────────┬───────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [91]:
db.query(f"""
    SELECT *
    FROM idiom_df
    WHERE idiom == 'to err is human, to forgive divine'
""").show()

db.query(f"""
    SELECT DISTINCT sources, entry
    FROM idiom_df
    WHERE entry == '""'
""").show()

┌────────────┬────────────────────────────────────┬───────────────────────────────┬─────────┬───────┬────────────┐
│ variations │               idiom                │            sources            │  entry  │  id   │ confidence │
│ varchar[]  │              varchar               │           varchar[]           │ varchar │ int64 │   int64    │
├────────────┼────────────────────────────────────┼───────────────────────────────┼─────────┼───────┼────────────┤
│ []         │ to err is human, to forgive divine │ [Oxford Dictionary of Idioms] │ ""      │  8245 │          1 │
│ []         │ to err is human, to forgive divine │ [Oxford Dictionary of Idioms] │ ""      │  8474 │          1 │
└────────────┴────────────────────────────────────┴───────────────────────────────┴─────────┴───────┴────────────┘

┌────────────────────────────────────────────────────┬─────────┐
│                      sources                       │  entry  │
│                     varchar[]                      │ varchar │

In [49]:
## CONFIDENCE

db.query(f"""
    SELECT *
    FROM idiom_df 
    WHERE confidence IS NULL
""").show()

db.query(f"""
    SELECT DISTINCT confidence, count(*)
    FROM idiom_df 
    GROUP BY confidence
""").show()

db.query(f"""
    WITH num_sources AS (
        SELECT id, len(sources) as len
        FROM idiom_df 
    )   
    SELECT df.id, ns.len, df.confidence, (ns.len == df.confidence) as matches
    FROM num_sources ns
    LEFT JOIN idiom_df df ON df.id == ns.id
    WHERE matches IS FALSE
""").show()

db.query(f"""
    SELECT *
    FROM idiom_df 
    WHERE id == 6674
""").show()

┌────────────┬─────────┬───────────┬─────────┬───────┬────────────┐
│ variations │  idiom  │  sources  │  entry  │  id   │ confidence │
│ varchar[]  │ varchar │ varchar[] │ varchar │ int64 │   int64    │
├────────────┴─────────┴───────────┴─────────┴───────┴────────────┤
│                             0 rows                              │
└─────────────────────────────────────────────────────────────────┘

┌────────────┬──────────────┐
│ confidence │ count_star() │
│   int64    │    int64     │
├────────────┼──────────────┤
│          2 │          873 │
│          3 │           90 │
│          1 │         8758 │
└────────────┴──────────────┘

┌───────┬───────┬────────────┬─────────┐
│  id   │  len  │ confidence │ matches │
│ int64 │ int64 │   int64    │ boolean │
├───────┼───────┼────────────┼─────────┤
│  6674 │     2 │          1 │ false   │
└───────┴───────┴────────────┴─────────┘

┌─────────────────────────────┬────────────────────────────┬───────────────────────────────────────────

In [ ]:
## SOURCES

db.query(f"""
    SELECT DISTINCT sources
    FROM idiom_df
""")

┌────────────────────────────────────────────────────────────────┐
│                            sources                             │
│                           varchar[]                            │
├────────────────────────────────────────────────────────────────┤
│ [Learn English Today]                                          │
│ [Wiktionary, Oxford Dictionary of Idioms]                      │
│ [Wiktionary, Oxford Dictionary of Idioms, Learn English Today] │
│ [Wiktionary]                                                   │
│ [Oxford Dictionary of Idioms]                                  │
│ [Oxford Dictionary of Idioms, Learn English Today]             │
│ [Wiktionary, Learn English Today]                              │
└────────────────────────────────────────────────────────────────┘

In [86]:
## ENTRY

db.query(f"""
    SELECT distinct df1.entry
    FROM idiom_df df1
    INNER JOIN idiom_df df2 ON df1.entry == df2.entry
""")

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
## ID

db.query(f"""
    WITH cte AS (
        SELECT DISTINCT id
        FROM idiom_df
         )
    SELECT count(*)
    FROM cte
""").show()

db.query(f"""
    SELECT min(id), max(id)
    FROM idiom_df
""").show()

db.query(f"""
WITH missing AS (
  SELECT id
  FROM generate_series(1, (SELECT MAX(id) FROM idiom_df)) AS t(id)
  EXCEPT
  SELECT id FROM idiom_df
),
grp AS (
  SELECT
    id,
    id - row_number() OVER (ORDER BY id) AS g
  FROM missing
)
SELECT
  MIN(id) AS start_id,
  MAX(id) AS end_id,
  COUNT(*) AS count_in_range
FROM grp
GROUP BY g
WHERE 
ORDER BY start_id;
""").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         9721 │
└──────────────┘

┌─────────┬─────────┐
│ min(id) │ max(id) │
│  int64  │  int64  │
├─────────┼─────────┤
│       1 │    9878 │
└─────────┴─────────┘

┌──────────┬────────┬────────────────┐
│ start_id │ end_id │ count_in_range │
│  int64   │ int64  │     int64      │
├──────────┼────────┼────────────────┤
│     8108 │   8108 │              1 │
│     8993 │   8993 │              1 │
│     9021 │   9021 │              1 │
│     9061 │   9061 │              1 │
│     9115 │   9115 │              1 │
│     9163 │   9309 │            147 │
│     9352 │   9352 │              1 │
│     9359 │   9359 │              1 │
│     9392 │   9392 │              1 │
│     9507 │   9507 │              1 │
│     9544 │   9544 │              1 │
├──────────┴────────┴────────────────┤
│ 11 rows                  3 columns │
└────────────────────────────────────┘



In [105]:
db.query(f"""
    SELECT DISTINCT sources
    FROM idiom_df 
    WHERE entry == '""'
""").show()

db.query(f"""
    SELECT DISTINCT sources
    FROM idiom_df 
    WHERE entry == '[]'
""").show()

┌────────────────────────────────────────────────────┐
│                      sources                       │
│                     varchar[]                      │
├────────────────────────────────────────────────────┤
│ [Learn English Today]                              │
│ [Oxford Dictionary of Idioms]                      │
│ [Oxford Dictionary of Idioms, Learn English Today] │
└────────────────────────────────────────────────────┘

┌───────────────────────────────────────────┐
│                  sources                  │
│                 varchar[]                 │
├───────────────────────────────────────────┤
│ [Wiktionary, Learn English Today]         │
│ [Wiktionary]                              │
│ [Wiktionary, Oxford Dictionary of Idioms] │
└───────────────────────────────────────────┘



In [111]:

print("sources with \"\"")
db.query(f"""
    SELECT DISTINCT sources
    FROM read_json_auto('{FILE_PATH}') 
    WHERE entry == '""'
""").show()

print("sources with []")
db.query(f"""
    SELECT DISTINCT sources
    FROM read_json_auto('{FILE_PATH}') 
    WHERE entry == '[]'
""").show()

print("null?")
db.query(f"""
    SELECT DISTINCT sources
    FROM read_json_auto('{FILE_PATH}') 
    WHERE entry IS NULL
""").show()

sources with ""
┌────────────────────────────────────────────────────┐
│                      sources                       │
│                     varchar[]                      │
├────────────────────────────────────────────────────┤
│ [Oxford Dictionary of Idioms]                      │
│ [Oxford Dictionary of Idioms, Learn English Today] │
│ [Learn English Today]                              │
└────────────────────────────────────────────────────┘

sources with []
┌───────────────────────────────────────────┐
│                  sources                  │
│                 varchar[]                 │
├───────────────────────────────────────────┤
│ [Wiktionary, Oxford Dictionary of Idioms] │
│ [Wiktionary, Learn English Today]         │
│ [Wiktionary]                              │
└───────────────────────────────────────────┘

null?
┌───────────┐
│  sources  │
│ varchar[] │
├───────────┤
│  0 rows   │
└───────────┘

